## Проект: вариант 1
Представьте, что вы работаете в компании, которая разрабатывает мобильные игры. К вам пришел менеджер с рядом задач по исследованию нескольких аспектов мобильного приложения:

1. В первую очередь, его интересует показатель retention. Напишите функцию для его подсчета.
2. Помимо этого, в компании провели A/B тестирование наборов акционных предложений. На основе имеющихся данных определите, какой набор можно считать лучшим и на основе каких метрик стоит принять правильное решение.
3. Предложите метрики для оценки результатов последнего прошедшего тематического события в игре.
 
### Задание 1
Retention – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока). Данные лежат в папке `shared` и имеют следующую структуру:

`shared/problem1-reg_data.csv` – данные о времени регистрации

|reg_ts|uid |
|------|----|
|906166566|2|
|906344325|3|
|906686169|4|
|906893386|5|
|906980227|6|

`shared/problem1-auth_data.csv` – данные о времени захода пользователей в игру

|auth_ts|uid|
|------|----|
|906166566|2|
|924422172|2|
|937374732|2|
|947425117|2|
|955630339|2|

Функция должна быть написана на python. В ходе решения можно тестировать работу функции как на полном датасете, так и на части (сэмпле) данных.

### Задание 2
Имеются результаты A/B теста, в котором двум группам пользователей предлагались различные наборы акционных предложений. Известно, что ARPU в тестовой группе выше на 5%, чем в контрольной. При этом в контрольной группе 1928 игроков из 202103 оказались платящими, а в тестовой – 1805 из 202667.

Какой набор предложений можно считать лучшим? Какие метрики стоит проанализировать для принятия правильного решения и как?

Формат [данных](https://disk.yandex.ru/d/SOkIsD5A8xlI7Q):

|user_id|revenue|testgroup|
|-------|-------|---------|
|   1   |   0   |    b    |
|   2   |   0   |    a    |
|   3   |   0   |    a    |
|   4   |   0   |    b    |
|   5   |   0   |    b    |

### Задание 3
В игре Plants & Gardens каждый месяц проводятся тематические события, ограниченные по времени. В них игроки могут получить уникальные предметы для сада и персонажей, дополнительные монеты или бонусы. Для получения награды требуется пройти ряд уровней за определенное время. С помощью каких метрик можно оценить результаты последнего прошедшего события?

Предположим, в другом событии мы усложнили механику событий так, что при каждой неудачной попытке выполнения уровня игрок будет откатываться на несколько уровней назад. Изменится ли набор метрик оценки результата? Если да, то как?

### Версия Jupyter Notebook
Версия Jupyter Notebook, на которой выполнялась работа - `6.5.7`

### Откроем данные и импортируем библиотеки

In [231]:
# импортируем библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pingouin as pg
from scipy.stats import mannwhitneyu

In [232]:
# прочитаем данные
try:
    reg_data = pd.read_csv('problem1-reg_data.csv', sep = ';')
    auth_data = pd.read_csv('problem1-auth_data.csv', sep = ';')
except:
    reg_data = pd.read_csv('shared/problem1-reg_data.csv', sep = ';')
    auth_data = pd.read_csv('shared/problem1-auth_data.csv', sep = ';')

In [233]:
print(f'Размер регистрационных данных составляет: {reg_data.shape[0]} строк и {reg_data.shape[1]} столбцов')
print(f'Размер авторизационных данных составляет: {auth_data.shape[0]} строк и {auth_data.shape[1]} столбцов')

Размер регистрационных данных составляет: 1000000 строк и 2 столбцов
Размер авторизационных данных составляет: 9601013 строк и 2 столбцов


In [234]:
reg_data.head()

,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3
3,959523541,4
4,969103313,5


In [235]:
auth_data.head()

,auth_ts,uid
0,911382223,1
1,932683089,2
2,932921206,2
3,933393015,2
4,933875379,2


In [236]:
reg_data.info()
print()
print()
auth_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   reg_ts  1000000 non-null  int64
 1   uid     1000000 non-null  int64
dtypes: int64(2)
memory usage: 15.3 MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9601013 entries, 0 to 9601012
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   auth_ts  int64
 1   uid      int64
dtypes: int64(2)
memory usage: 146.5 MB


### Выводы:
Представлены 2 датасета: данные о времени регистрации и данные о времени захода пользователя в игру.

В данных о времени регистрации содержится 1 000 000 строк и 2 столбца (id и время регистрации пользователя).
В данных о времени авторизации содержится 9 601 013 строк и 2 столбца (id и время авторизации пользователя).

Поля `reg_ts` и `auth_ts` в формате `int64`, потребуется привести данные в формат `timestamp`.
Также проверим регистрационные данные на дубли `id`, чтобы не было повторений.

### Предобработка данных

In [237]:
# посмотрим данные на наличие дублей
print(f'Количество дублей id в регистрационных данных равно:\
 {reg_data.uid.duplicated().sum()}')

Количество дублей id в регистрационных данных равно: 0


In [238]:
# изменим тип данных полей reg_ts и auth_ts
# время и даты уберем
reg_data['reg_ts'] = pd.to_datetime(reg_data['reg_ts'], unit='s').dt.date
reg_data.head()

,reg_ts,uid
0,1998-11-18,1
1,1999-07-22,2
2,2000-01-13,3
3,2000-05-28,4
4,2000-09-16,5


In [239]:
auth_data['auth_ts'] = pd.to_datetime(auth_data['auth_ts'], unit='s').dt.date
auth_data.head()

,auth_ts,uid
0,1998-11-18,1
1,1999-07-22,2
2,1999-07-25,2
3,1999-07-31,2
4,1999-08-05,2


### Задание 1. Напишем функцию, которая будет считать retention игроков (по дням от даты регистрации игрока)

In [240]:
def retention_per_day(reg_df, auth_df, reg_date, period, as_percentage=True):
    reg_date = pd.to_datetime(reg_date)
    
    # сделаем преобразование строки в дату
    reg_df['reg_ts'] = pd.to_datetime(reg_df['reg_ts'])
    auth_df['auth_ts'] = pd.to_datetime(auth_df['auth_ts'])

    # выберем когорту по дате регистрации
    cohort_users = reg_df[reg_df['reg_ts'] == reg_date][['uid', 'reg_ts']]
    cohort_size = cohort_users['uid'].nunique()

    if cohort_size == 0:
        return pd.DataFrame([{'cohort_size': 0,\
                              **{f'day_{i}': 0 for i in range(period + 1)}}])

    # Слияние и расчет разницы по дням
    merged = pd.merge(auth_df, cohort_users, on='uid', how='inner')
    merged['day'] = (merged['auth_ts'] - merged['reg_ts']).dt.days
    merged = merged[(merged['day'] >= 0) & (merged['day'] <= period)]
    merged = merged.drop_duplicates(subset=['uid', 'day'])

    # Расчет retention
    retention_counts = merged.groupby('day')['uid'].nunique()
    retention = {'cohort_size': cohort_size}
    for day in range(period + 1):
        count = retention_counts.get(day, 0)
        value = round(count / cohort_size * 100, 2) if as_percentage else count
        retention[f'Day_{day}'] = value

    return pd.DataFrame([retention])

Протестируем, выбрав даты из регистрационных данных. Выберем пользователя под id 108, который заходил в приложение несколько раз после регистрации

In [241]:
# увеличим просматриваемость столбцов
pd.set_option('display.max_columns',40)

# протестируем функцию на данных
result = retention_per_day(reg_data, auth_data, "2005-05-30", 7)
display(result)

,cohort_size,Day_0,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7
0,1,100.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0


Теперь также протестируем, выбрав рандомную датую.

In [242]:
result_2 = retention_per_day(reg_data, auth_data, "2000-03-03", 30)
display(result_2)

,cohort_size,day_0,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Вывод:
Была настроена функция, которая считает `retention` игроков по дням от даты регистрации. 

В функции необходимо указать данные о времени регистрации, данные о времени захода пользователей в игру, дату когорты и гранулярность (в днях).

### Задание 2. Определим какой набор предложений можно считать лучшим и какие метрики стоит проанализировать для принятия правильного решения

Для начала нам необходимо проверить корректность переданной информации о том, что APRU в тестовой группе выше на 5%, чем в контрольной.

ARPU - это средняя выручка на пользователя. Но ее недостаточно, чтобы сделать вывод по проведенному тесту. Посмотрим на другие метрики, которые помогут подтвердить или опровергнуть результаты A/B теста.

Используем следующие метрики:
- ARPPU (Average Revenue Per Paying User) - средний доход на платящего пользователя;
- Конверсия в платящего пользователя;
- LTV (Lifetime value) - долгосрочная ценность пользователей.

In [243]:
ab_test = pd.read_csv('Проект_1_Задание_2.csv', sep=';')
print(f'Размер данных составляет: {ab_test.shape[0]} строк и {ab_test.shape[1]} столбца')

Размер данных составляет: 404770 строк и 3 столбца


In [244]:
# посмотрим на часть данных
ab_test.head()

,user_id,revenue,testgroup
0,1,0,b
1,2,0,a
2,3,0,a
3,4,0,b
4,5,0,b


In [245]:
# перепроверим полученные данные
# правда ли ARPU на 5% выше в тестовой группе
users_a = ab_test.query('testgroup == "a"')['user_id'].nunique()
users_b = ab_test.query('testgroup == "b"')['user_id'].nunique()

sum_a = ab_test.query('testgroup == "a"')['revenue'].sum()
sum_b = ab_test.query('testgroup == "b"')['revenue'].sum()

arpu_a = sum_a / users_a
arpu_b = sum_b / users_b

diff_arpu = (arpu_b / arpu_a - 1) * 100
print(f"ARPU группы B выше на {round(diff_arpu, 2)}%") 

ARPU группы B выше на 5.26%


Данные подтвердились. В тестовой группе выше на 5%, чем в контрольной.

In [246]:
# проверим также кол-во платящих пользователей
group_a = ab_test.query('testgroup == "a"')
print('Количество платящих пользователей в группе A:',\
      group_a[group_a['revenue'] != 0]['user_id'].count())

group_b = ab_test.query('testgroup == "b"')
print('Количество платящих пользователей в группе B:',\
      group_b[group_b['revenue'] != 0]['user_id'].count())

Количество платящих пользователей в группе A: 1928
Количество платящих пользователей в группе B: 1805


In [247]:
# рассчитаем конверсию на платящего пользователя по группам
grouped_df = ab_test.groupby('testgroup').agg(
             users=('user_id', 'count'),
             payers=('revenue', lambda x: (x > 0).sum())
).reset_index()

grouped_df['CR'] = grouped_df['payers'] / grouped_df['users'] * 100
display(grouped_df)

,testgroup,users,payers,CR
0,a,202103,1928,0.953969
1,b,202667,1805,0.890624


Как и выяснилось ранее, количество платящих клиентов в группе B стало на 6% меньше.

In [248]:
# рассчитаем метрику ARPPU
# общий доход группы делим на число платящих клиентов
paying_users_a = group_a[group_a['revenue'] > 0]
paying_users_b = group_b[group_b['revenue'] > 0]

arppu_a = paying_users_a['revenue'].sum() / paying_users_a.shape[0]
arppu_b = paying_users_b['revenue'].sum() / paying_users_b.shape[0]

print(f'ARPPU группы A: {arppu_a:.2f}')
print(f'ARPPU группы B: {arppu_b:.2f}')

ARPPU группы A: 2664.00
ARPPU группы B: 3003.66


В контрольной группе средний доход на платящего пользователя выше.

Из этого следует, что выводы по ARPU корректны, а также при меньшем количестве платящих пользователей, средний доход на платящего пользователя выше, что может говорить о

### Проведем статистические тесты

Для начала необходимо определиться какие статистические тесты будем использовать:

* **CR в платящего пользователя можно оценить при помощи Z-теста**\
Выбираем Z-тест, так как сравниваем доли в двух независимых группах.
* **ARPU - тест Манна-Уитни**\
Выбираем тест Манна-Уитни, так как не чувствителен к выбросам, а в данных много нулей 
* **ARPPU - T-тест**\
Выбираем для проверки разницы средних. Также подходит, если распределение близко к нормальному.

#### Z-тест на сравнение CR

In [249]:
from statsmodels.stats.proportion import proportions_ztest

payers = [paying_users_a.shape[0], paying_users_b.shape[0]]
totals = [group_a.shape[0], group_b.shape[0]]

z_stat, pval = proportions_ztest(payers, totals)
print(f"P-value, проведенного теста, равен: {pval:.4f}")

P-value, проведенного теста, равен: 0.0350


#### Вывод по z-тесту
Так как `p-value < 0.05`, мы можем отвергнуть нулевую гипотезу о равенстве конверсий.

Следовательно, **разница в CR между группами статистически значима**. Конверсия в группе А выше, чем в группе В.

Это означает, что в контрольной группе пользователи хуже конвертируются в платящих, но при этом ARPU у контрольной группы выше на 5%.

#### Тест Манна-Уитни

In [250]:
stat, p_value = mannwhitneyu(
    group_a['revenue'],
    group_b['revenue'],
    alternative='two-sided'
)

print(f"Статистика U-теста: {stat:.2f}")
print(f"P-value: {p_value:.4f}")

if p_value < 0.05:
    print("Разница в ARPU между группами статистически значима (p < 0.05).")
else:
    print("Нет статистически значимой разницы в ARPU между группами (p ≥ 0.05).")

Статистика U-теста: 20491259376.00
P-value: 0.0627
Нет статистически значимой разницы в ARPU между группами (p ≥ 0.05).


#### Вывод:
Контрольная группа (группа b), несмотря на 5% к ARPU, не демонстрирует статистически значимого улучшения.

В текущих условиях недостаточно оснований для перехода на группу B.

#### T-test (метрика ARPPU)

In [251]:
ttest_results = pg.ttest(
    paying_users_a['revenue'],
    paying_users_b['revenue'],
    alternative='two-sided'
)

display(ttest_results)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.64463,1943.481522,two-sided,0.100208,"[-744.7, 65.38]",0.052132,0.142,0.356381


#### Вывод:
Статистически значимой разницы в ARPPU между контрольной и тестовой группами нет.

Из этого следует, что доход на пользователя примерно одинаков в обеих группах. Мы не можем рекомендовать тестовую группу B на основе ARPPU.

### Общий вывод:

На основе проведенного анализа можно сделать следующий вывод:

**Лучшим можно считать набор предложений из группы A.**

1. Конверсия (CR) в платящих в группе A значимо выше, чем в B.
2. ARPU группы B выше на 5%,как и было сказано, но эта разница не статистически значима (проверено статистическим тестом Манна-Уитни).
3. ARPPU (средний доход с платящего пользователя) также не отличается значимо между группами (проверено t-test).

Таким образом, при равном доходе с платящих и более высокой конверсии, группа A обеспечивает большее общее число покупателей и, как следствие, более стабильную выручку.

### Задание 3. С помощью каких метрик можно оценить результаты последнего прошедшего события? Изменится ли набор метрик оценки результата, если мы усложним механику? Если да, то как?

Для начала ответим на базовый вопрос - с помощью каких метрик можно оценить результаты последнего прошедшего события (без откатов):

1. Конверсия (доля пользователей, начавших событие) - поможет оценить интерес к проводимому событию

$$
\text{Конверсия} = \frac{\text{Количество пользователей, выполнивших событие}}{\text{Общее количество пользователей}}
$$

2. Процент игроков, прошедших все уровни (использовать дневную гранулярность). Позволит оценить сложность того или иного уровня и где пользователи теряют инетерес.
3. Retention - позволит оценить как часто пользователи проходят событие на протяжении месяца, возращаются ли они получать бонусы или может мотивации недостаточно.

Если мы усложним механику событий так, что при каждой неудачной попытке выполнения уровня игрок будет откатываться на несколько уровней назад, то следует использовать больше метрик:

1. Churn - позволит понять, покидают ли пользователи игру после серии неудач
2. Среднее количество попыток на уровень (доля неудач на каждом уровне) - позволит понять сложность уровня

Следовательно, можно подвести основной итог, что в 1 сценарии пользователь не будет испытывать сильных сложностей на пути прохождения (только лутать бонусы). Основными метриками будут: участие, прогресс, завершение, удержание, полученные призы.

Во 2-м сценарии проверяется стойкость пользователя. Основными метриками будут: как в 1 сценарии и дополнительно отказ и отток пользователей. В случае усложнённой механики нужно особенно внимательно следить за тем, не вызывает ли она отток или снижение удовлетворённости и, возможно, даже произвести дополнительные траты на службу поддержки, которое придется отвечать на гневные отзывы.